<a href="https://colab.research.google.com/github/shivamdhumal77/LLM_Projects/blob/main/llm_sound_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
!pip install git+https://github.com/openai/whisper.git
!pip install parler-tts
!pip install transformers
!pip install langchain-community
!pip install soundfile
 # If you want to test audio playback functionality, though note it's not ideal for Colab.


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-l680hktf
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-l680hktf
  Resolved https://github.com/openai/whisper.git to commit 90db0de1896c23cbfaf0c58bc2d30665f709f170


In [ ]:
pip install git+https://github.com/huggingface/parler-tts.git

In [ ]:
pip install colab-xterm

In [ ]:
%load_ext colabxterm

In [ ]:
# put this in the terminal and follow these steps
#curl https://ollama.ai/install.sh | sh
# ollama serve & ollama pull llama3.2
%xterm

In [ ]:
%xterm

In [ ]:
pip install ollama

In [ ]:
%xterm

In [ ]:
import whisper
import torch
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer, set_seed
import numpy as np
from langchain_community.llms.ollama import Ollama
import soundfile as sf
from IPython.display import Audio

# Initialize Whisper model for transcription
whisper_model = whisper.load_model("tiny")  # Use 'tiny' for faster transcription

# Initialize ParlerTTS for speech synthesis
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model_name = "parler-tts/parler-tts-mini-v1"  # Adjust this model name as needed
tts_model = ParlerTTSForConditionalGeneration.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Initialize Ollama for text-based response generation
ollama = Ollama(model="llama3.2")

# Function for transcribing uploaded audio file using Whisper
def transcribe_from_audio(file_path):
    audio = whisper.load_audio(file_path)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(device)

    options = whisper.DecodingOptions(fp16=False)
    result = whisper.decode(whisper_model, mel, options)
    return result.text

# Function for generating a response from Ollama
def generate_response(user_input):
    print(f"User: {user_input}")
    response = ollama(user_input)
    response_text = response.strip()  # Clean up the response
    print(f"Ollama: {response_text}")
    return response_text

# Function to synthesize speech using ParlerTTS
def synthesize_speech(text, voice_description):
    input_ids = tokenizer(voice_description, return_tensors="pt").input_ids.to(device)
    prompt_input_ids = tokenizer(text, return_tensors="pt").input_ids.to(device)
    generation = tts_model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
    audio_arr = generation.cpu().numpy().squeeze()
    return audio_arr

# Function to play audio using Colab's native player
def play_audio(audio_array, sample_rate=22050):
    sf.write("synthesized_audio.wav", audio_array, sample_rate)  # Save audio to a .wav file
    return Audio("synthesized_audio.wav", autoplay=True)  # Return an Audio object for playback in Colab

# Main function for simulating the conversation
def conversation(file_path):
    voice_description = "A female speaker delivers a slightly expressive and animated speech with a moderate speed and pitch. The recording is of very high quality, with the speaker's voice sounding clear and very close up"

    # Step 1: Transcribe user input from the uploaded audio file
    user_input = transcribe_from_audio(file_path)

    # Step 2: Generate a response using Ollama
    response = generate_response(user_input)

    # Step 3: Synthesize response into speech
    audio = synthesize_speech(response, voice_description)

    # Step 4: Play synthesized speech
    return play_audio(audio)


# Upload an audio file manually in Colab, e.g., using this code snippet:
from google.colab import files
uploaded = files.upload()

# Choose the first uploaded file to use for transcription
file_path = list(uploaded.keys())[0]

# Run the conversation (transcription, response generation, and playback)
conversation(file_path)



/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
/usr/local/li

In [ ]:
import torch
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler-tts-mini-v1").to(device)
tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler-tts-mini-v1")

prompt = "Hey, how are you doing today?"
description = "A female speaker delivers a slightly expressive and animated speech with a moderate speed and pitch. The recording is of very high quality, with the speaker's voice sounding clear and very close up."

input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
audio_arr = generation.cpu().numpy().squeeze()
sf.write("parler_tts_out.wav", audio_arr, model.config.sampling_rate)